# 量子回路からのサンプリング

先ほどの演習では特定の量子ビットに着目して、測定を行う手法を学びました。

このノートブックでは、これを発展させ、一つずつ順番に量子ビットを測定していき、値を確定させていきながらサンプリングを行う方法について学びます。

まずは、GHZ状態を作る簡単な回路を題材に実装します。

In [ ]:
!pip install tensornetwork

In [ ]:
import tensornetwork as tn
import numpy as np
import random
import copy

まずは、GHZ状態を作る回路を表現するテンソルネットワークを作成します。

この内容は最初の演習とほとんど同様です。

In [ ]:
h_gate = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

cx_gate = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 0, 1],
                    [0, 0, 1, 0]])

In [ ]:
def make_initial_nodes(n_qubits):
    initial_nodes = list()

    for i in range(n_qubits):
        initial_nodes.append(tn.Node(np.array([1, 0])))

    return initial_nodes

In [ ]:
def make_gate_nodes(n_qubits):
    gate_nodes = list()

    gate_nodes.append(tn.Node(h_gate))

    for i in range(n_qubits - 1):
        gate_nodes.append(tn.Node(cx_gate.reshape(2,2,2,2)))

    return gate_nodes

In [ ]:
def make_edge(n_qubits, initial_state_nodes, gate_nodes):
    # 1量子ビット目の初期状態から、Hゲートの入力
    tn.connect(initial_state_nodes[0][0], gate_nodes[0][0])

    # Hゲートの出力から、1個目のCXゲートの入力の1量子ビット目
    tn.connect(gate_nodes[0][1], gate_nodes[1][0])

    # i個目のCXゲートの出力の1量子ビット目から、i+1個目のCXゲートの入力の1量子ビット目
    for i in range(1, n_qubits - 1):
        tn.connect(gate_nodes[i][2], gate_nodes[i+1][0])

    # i個目の量子ビットの初期状態から、i-1個目のCXゲートの入力の2量子ビット目
    for i in range(1, n_qubits):
        tn.connect(initial_state_nodes[i][0], gate_nodes[i][1])

次に、これを双方向のネットワークにして、量子ビットを一つ測定するような機能を作成します。

In [ ]:
def sample_bit(n_qubits, target_bit, sample):
    # 各種ノードの定義
    initial_nodes = make_initial_nodes(n_qubits)
    gate_nodes = make_gate_nodes(n_qubits)

    initial_nodes_r = make_initial_nodes(n_qubits)
    gate_nodes_r = make_gate_nodes(n_qubits)

    # ネットワークの結合
    make_edge(n_qubits, initial_nodes, gate_nodes)
    make_edge(n_qubits, initial_nodes_r, gate_nodes_r)

    if len(sample) == 0:
        for i in range(1, n_qubits):
            tn.connect(gate_nodes[i][3], gate_nodes_r[i][3])
        nodes = (initial_nodes + gate_nodes + initial_nodes_r + gate_nodes_r)
        output_edges = []
        output_edges.append(gate_nodes[-1][2])
        output_edges.append(gate_nodes_r[-1][2])

        result = tn.contractors.auto(nodes=nodes, output_edge_order=output_edges)
        result_tensor = result.tensor

    else:
        # すでに値が確定しているビットについては、それに相当するノードを追加する
        observed_nodes = list()
        for i in range(len(sample)):
            if sample[i] == '0':
                observed_nodes.append(tn.Node(np.array([1, 0])))
            else:
                observed_nodes.append(tn.Node(np.array([0, 1])))
        observed_nodes_r = copy.deepcopy(observed_nodes)

        tn.connect(gate_nodes[-1][2], observed_nodes[0][0])
        tn.connect(gate_nodes_r[-1][2], observed_nodes_r[0][0])

        for i in range(1, len(sample)):
            tn.connect(gate_nodes[i][3], observed_nodes[i][0])
            tn.connect(gate_nodes_r[i][3], observed_nodes_r[i][0])

        for i in range(len(sample)+1, n_qubits):
            tn.connect(gate_nodes[i][3], gate_nodes_r[i][3])

        nodes = (initial_nodes + gate_nodes + initial_nodes_r + gate_nodes_r + observed_nodes + observed_nodes_r)
        output_edges = []
        output_edges.append(gate_nodes[target_bit][3])
        output_edges.append(gate_nodes_r[target_bit][3])

        result = tn.contractors.auto(nodes=nodes, output_edge_order=output_edges)
        result_tensor = result.tensor

    sampling_weights = [result.tensor[0][0], result.tensor[1][1]]
    return sampling_weights

ためしに、一つ量子ビットのサンプリングを実施してみます。

In [ ]:
# 0番の量子ビットのサンプリング
sample_bit(3, 0, "")

In [ ]:
# 0番の量子ビットが"0"のときに、1番の量子ビットをサンプリング
sample_bit(3, 1, "0")

最後に、回路全体からのサンプリングを実装します。

1量子ビットずつサンプリングを行い、得られた確率分布に基づいて、確率的に値を確定させていきます。

また、同じ結果については再計算する必要がないので、辞書に保存しておくようにします。

In [ ]:
def sample_circuit(n_qubits, n_shots=10):
    sample_list = list() # 得られたサンプルを保存するためのリスト
    sampling_weights_dict = dict() # 一度計算した結果を記録しておくための辞書

    for _ in range(n_shots):
        sample = ""

        for i in range(n_qubits):
            if sample in sampling_weights_dict.keys(): # 計算済みの結果であれば、読み出す
                sampling_weights = sampling_weights_dict[sample]
            else:
                sampling_weights = sample_bit(n_qubits, i, sample)
                sampling_weights_dict[sample] = sampling_weights # 結果を保存しておく

            sampling_bit = random.choices([0, 1], k=1, weights=sampling_weights) # 得られた結果から、確率的にビットの値を決定する
            sample += str(sampling_bit[0])

        sample_list.append(sample)

    return sample_list

結果を確認してみます。

In [ ]:
%%time
sample_circuit(30)